# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import gmaps
import os
import json

# Import API key
from api_keys import g_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,beckley,37.7782,-81.1882,4.34,74,0,0.45,US,1681199752
1,1,drawsko pomorskie,53.5306,15.8097,12.72,76,100,2.24,PL,1681199753
2,2,airai,-8.9266,125.4092,19.51,90,63,0.98,TL,1681199754
3,3,busselton,-33.6500,115.3333,20.89,57,50,4.49,AU,1681199756
4,4,mataura,-46.1927,168.8643,11.38,97,83,1.15,NZ,1681199757


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
locations = city_data_df[["Lat", "Lng"]].astype(float)
humidity = city_data_df["Humidity"]

gmaps.configure(api_key = g_key)

# Display the map and creating a heat map
map_display = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=5)

map_display.add_layer(heat_layer)
map_display

Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
#city_data_df.info()

vacation_cities_df = city_data_df.loc[(city_data_df["Max Temp"] > 15)
                                    & (city_data_df["Max Temp"] < 23)
                                    & (city_data_df["Humidity"] < 60)]

# Display sample data
vacation_cities_df.head()

vacation_cities_df.shape

(34, 10)

In [7]:
vacation_cities_clean = vacation_cities_df.dropna()
vacation_cities_clean.shape

(31, 10)

In [8]:
vacation_cities_clean.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,busselton,-33.6500,115.3333,20.89,57,50,4.49,AU,1681199756
42,42,abha,18.2164,42.5053,22.14,27,20,8.23,SA,1681199806
46,46,mount gambier,-37.8333,140.7667,15.94,54,94,3.03,AU,1681199812
56,56,east london,-33.0153,27.9116,18.52,45,0,10.80,ZA,1681199822
58,58,port alfred,-33.5906,26.8910,18.99,53,39,4.92,ZA,1681199825


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_cities_clean.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# Display sample data

hotel_df["Hotel Name"] =""
hotel_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,3,busselton,-33.6500,115.3333,20.89,57,50,4.49,AU,1681199756,
42,42,abha,18.2164,42.5053,22.14,27,20,8.23,SA,1681199806,
46,46,mount gambier,-37.8333,140.7667,15.94,54,94,3.03,AU,1681199812,
56,56,east london,-33.0153,27.9116,18.52,45,0,10.80,ZA,1681199822,
58,58,port alfred,-33.5906,26.8910,18.99,53,39,4.92,ZA,1681199825,
62,62,bredasdorp,-34.5322,20.0403,16.35,58,74,2.28,ZA,1681199829,
111,111,saint george,37.1041,-113.5841,17.33,34,0,1.54,US,1681199887,
171,171,beyneu,45.3167,55.2000,19.91,30,100,2.34,KZ,1681199960,
175,175,dasoguz,41.8363,59.9666,17.97,55,0,4.12,TM,1681199965,
195,195,port elizabeth,-33.9180,25.5701,19.50,52,0,9.26,ZA,1681199988,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
parameters = {"radius" : 5000, "types" : "hotel", "keyword" : "hotels", "key" : g_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for x, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    latitude = row["Lat"]
    longitude = row["Lng"]
    params["location"] = str(latitude) + "," + str(longitude)
    hotel_name = requests.get(url, params=parameters).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[x, "Hotel Name"] = hotel_name["results"][0]["name"]
    except:
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[x, "Hotel Name"] = "N/A"
        print("No hotels near " + str(row["City"]))
        
    # Log the search results
    print(f"{hotel_df.loc[x, 'City']} - nearest hotel: {hotel_df.loc[x, 'Hotel Name']}")

# Display sample data
hotel_df


# My Google Maps API is not working for some reason, so not hotels are coming up. 
# Unsure how to fix it at this time>

Starting hotel search
No hotels near busselton
busselton - nearest hotel: N/A
No hotels near abha
abha - nearest hotel: N/A
No hotels near mount gambier
mount gambier - nearest hotel: N/A
No hotels near east london
east london - nearest hotel: N/A
No hotels near port alfred
port alfred - nearest hotel: N/A
No hotels near bredasdorp
bredasdorp - nearest hotel: N/A
No hotels near saint george
saint george - nearest hotel: N/A
No hotels near beyneu
beyneu - nearest hotel: N/A
No hotels near dasoguz
dasoguz - nearest hotel: N/A
No hotels near port elizabeth
port elizabeth - nearest hotel: N/A
No hotels near zhezkazgan
zhezkazgan - nearest hotel: N/A
No hotels near codrington
codrington - nearest hotel: N/A
No hotels near agadir
agadir - nearest hotel: N/A
No hotels near sonoita
sonoita - nearest hotel: N/A
No hotels near sayyan
sayyan - nearest hotel: N/A
No hotels near san juan
san juan - nearest hotel: N/A
No hotels near miyako
miyako - nearest hotel: N/A
No hotels near ponta do sol
pont

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,3,busselton,-33.6500,115.3333,20.89,57,50,4.49,AU,1681199756,N/A
42,42,abha,18.2164,42.5053,22.14,27,20,8.23,SA,1681199806,N/A
46,46,mount gambier,-37.8333,140.7667,15.94,54,94,3.03,AU,1681199812,N/A
56,56,east london,-33.0153,27.9116,18.52,45,0,10.80,ZA,1681199822,N/A
58,58,port alfred,-33.5906,26.8910,18.99,53,39,4.92,ZA,1681199825,N/A
62,62,bredasdorp,-34.5322,20.0403,16.35,58,74,2.28,ZA,1681199829,N/A
111,111,saint george,37.1041,-113.5841,17.33,34,0,1.54,US,1681199887,N/A
171,171,beyneu,45.3167,55.2000,19.91,30,100,2.34,KZ,1681199960,N/A
175,175,dasoguz,41.8363,59.9666,17.97,55,0,4.12,TM,1681199965,N/A
195,195,port elizabeth,-33.9180,25.5701,19.50,52,0,9.26,ZA,1681199988,N/A


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE